# Zadání: Genomová vizualizace - Nautilus pompilius


# 📋 Pravidla práce
- Povoleno používat AI (ChatGPT, Claude, Copilot, atd.)

- Musíte rozumět každému řádku kódu - AI je asistent, ne řešitel

- Hodnotí se originalita a kvalita vizualizací

**Odevzdejte: Jupyter notebook přejmenovaný na JMENO_PRIJMENI_Genomova_vizualizace.ipynb**

# 🧬 Dataset
- Nautilus pompilius (hlavonožec, "živá fosílie")

- Soubor: Nautilus_pompilius_student.fasta (~30 MB)

- Obsahuje: ~8 scaffoldů (genomových fragmentů)

**Formát: Softmasked FASTA (malá/velká písmena, N pro neznámé báze)**

# 🎯 ÚKOLY
## ÚKOL 1: Základní explorace dat (15 bodů)

**Cíl: Vytvořit informativní dashboard o struktuře genomu.**

1.1 Statistika scaffoldů
Vytvořte funkci, která vrací DataFrame se statistikami:

```python
# Očekávaný výstup:
   Scaffold_ID     Délka(bp)    GC%     N%      A%      T%      C%      G%
0  >JABBOT010000001.1  1,250,345   42.3   1.2    28.1    27.4    21.5    22.8
1  >JABBOT010000002.1    893,456   38.7   0.8    30.2    29.1    19.3    20.6
1.2 Top 10 vizualizace
```
**Vytvořte multi-panelový graf (2x2):**

- A: Bar plot 10 nejdelších scaffoldů (délka v Mb)

- B: Pie chart distribuce délek (kategorie: <100kb, 100kb-1Mb, >1Mb)

- C: Scatter plot délka vs. GC obsah

- D: Heatmap korelace mezi A,T,G,C,N obsahy

**Formátovací požadavky:**

- Použijte plt.style.use('seaborn-v0_8-darkgrid')

- Přidejte anotace k outlierům

- Použijte logaritmické škály tam, kde to dává smysl

- Uložte graf jako scaffold_stats.png (300 DPI)

## ÚKOL 2: Analýza v oknech - GC a N profily (25 bodů)
### Cíl: Analyzovat distribuci nukleotidů podél genomu.

**2.1 Sliding windows implementace**

**Implementujte 3 různé algoritmy pro sliding windows:**

- **Naivní** - počítá každé okno zvlášť

- **Kumulativní suma** - optimalizovaný přístup

- **NumPy vektorizovaný** - np.lib.stride_tricks.sliding_window_view

**Funkce rozhraní:**

```python

def analyze_windows(sequence, window_size=10000, step_size=5000, 
                    method='cumulative'):
    """Vrací DataFrame s [position, gc_content, n_content, complexity]"""
```

**2.2 Interaktivní dashboard**
Vytvořte interaktivní vizualizaci pomocí *ipywidgets*:

```python
import ipywidgets as widgets
from IPython.display import display

# Widgety pro kontrolu parametrů
window_slider = widgets.IntSlider(value=10000, min=1000, max=50000, step=1000)
step_slider = widgets.IntSlider(value=5000, min=100, max=25000, step=500)
method_radio = widgets.RadioButtons(options=['naive', 'cumulative', 'numpy'])
scaffold_dropdown = widgets.Dropdown(options=scaffold_list)

@widgets.interact
def update_plot(window_size=window_slider, step_size=step_slider, 
                method=method_radio, scaffold=scaffold_dropdown):
```

## Dynamické překreslování grafu

**Výstup: 3-panelový graf s:**

- Průběhem GC% s pohyblivým průměrem (window=5)

- Průběhem N% s detekcí peaků (scipy.signal.find_peaks)

- Spectrogramem GC obsahu (pomocí plt.specgram)

## ÚKOL 3: Srovnání regionů a detekce patternů (30 bodů)
**Cíl: Identifikovat zajímavé genomové regiony.**

### 3.1 Detekce GC-bohatých a GC-chudých oblastí
Implementujte algoritmus pro:

- GC-bohaté oblasti: >65% GC, délka >5kb

- GC-chudé oblasti: <35% GC, délka >5kb

- N-bohaté oblasti: >10% N (sekvenční mezery)

**Výstup: DataFrame s anotovanými regiony:**


```python
   Region_ID  Scaffold          Start      End    Length   Type      GC%    N%
0  R1        >JABBOT...1      45,230   52,890    7,660    GC-rich   68.4   0.2
1  R2        >JABBOT...1     125,340  135,210    9,870    GC-poor   32.1   0.5
```

## 3.2 Genomový browser - mini verze
Vytvořte genomový browser pro jeden scaffold:

```python
def genomic_browser(scaffold_id, start=0, end=None, regions=None):
    """
    Vykreslí genomovou oblast s anotacemi.
    
    Vrstvy (z=index):
    0. GC obsah (barvy: modrá škála)
    1. N obsah (červená škála, průhlednost)
    2. GC-bohaté oblasti (zelené boxy)
    3. GC-chudé oblasti (červené boxy)
    4. Zoom window (interaktivní - `mpld3`)
    5. Anotace (popisky regionů)
    """
```

**Požadavky na vizualizaci:**

- Použijte matplotlib.gridspec pro komplexní layout

- Přidejte colorbar pro GC škálu

- Implementujte tooltips s mpld3 nebo mplcursors

- Umožněte zoom/pan pomocí *%matplotlib widget*

## 3.3 K-merová analýza regionů
**Pro každý typ regionu (GC-rich, GC-poor):**

- Extrahujte sekvence

- Spočítejte frekvenci 6-merů

- Najděte charakteristické motivy

- Vytvořte logo plot pomocí logomaker

```python
import logomaker as lm

# Vytvoření logo plotu pro GC-bohaté oblasti
gc_rich_logo = lm.Logo(ppm_matrix, color_scheme='classic')
gc_rich_logo.style_spines(visible=False)
gc_rich_logo.ax.set_title("GC-bohaté oblasti - 6-mer motify")
```

## ÚKOL 4: Pokročilé vizualizace (30 bodů)
**Cíl: Vytvořit publikovatelné vizualizace.**

## 4.1 Circos plot - inspirace
**Vytvořte zjednodušený Circos plot znázorňující:**

- **Vnější kruh:** Scaffoldy (úseky podle délky)

- **První vnitřní vrstva:** GC obsah (barva podle hodnoty)

- **Druhá vnitřní vrstva:** N obsah (tloušťka podle hodnoty)

- **Spojnice:** Homologie mezi scaffoldy (na základě společných k-merů)

### Implementace:

```python
import matplotlib.patches as mpatches

def create_circos_plot(scaffolds_data, connections=None):
    # Použijte mpatches.Wedge pro segmenty
    # Použijte ConnectionPatch pro spojnice
    # Použijte colormap pro GC hodnoty
```

## 4.2 3D vizualizace k-merového prostoru
**Projektujte 6-merové frekvence do 3D prostoru pomocí PCA:**

```python
from sklearn.decomposition import PCA
import plotly.graph_objects as go

# PCA redukce na 3D
pca = PCA(n_components=3)
kmer_3d = pca.fit_transform(kmer_matrix)

# 3D scatter plot s plotly
fig = go.Figure(data=[go.Scatter3d(
    x=kmer_3d[:,0], y=kmer_3d[:,1], z=kmer_3d[:,2],
    mode='markers',
    marker=dict(size=5, color=gc_values, colorscale='Viridis'),
    text=scaffold_names
)])
fig.update_layout(title="3D PCA k-merového prostoru")
```

## 4.3 Interaktivní dashboard s Plotly Dash/Bokeh
**Vytvořte webový dashboard s:**

- Výběrem scaffoldu (dropdown)

- Nastavením parametrů sliders

- Více propojenými grafy (linked brushing)

- Exportem do PNG/PDF

```python
# Ukázka s Bokeh
from bokeh.plotting import figure, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select, Slider

# Interaktivní zdroj dat
source = ColumnDataSource(data=dict(x=positions, y=gc_values))

# Widgety
scaffold_select = Select(title="Scaffold:", options=scaffold_list, value=scaffold_list[0])
window_slider = Slider(start=1000, end=50000, value=10000, step=1000, title="Window size")

# Callback pro aktualizaci
def update_data(attrname, old, new):
    # Aktualizuj data podle widgetů
    source.data = new_data

scaffold_select.on_change('value', update_data)
window_slider.on_change('value', update_data)
```

# 📊 HODNOCENÍ
## Kritéria:
- **Funkčnost (40%):** Všechny grafické výstupy fungují

- **Kvalita kódu (20%):** Čistý, komentovaný, efektivní kód

- **Kreativita vizualizací (25%):** Originalita a informační hodnota

- **Dokumentace (15%):** Komentáře a popis metod

## Bonusové body (+10):
- **Animace:** Time-lapse změn při různé velikosti okna

- **Webová aplikace:** Flask/Dash aplikace s dashboardem

- **Machine learning:** Klastrování scaffoldů podle kompozice

- **Optimalizace:** 10× rychlejší než baseline implementace

# 🎓 CO SE NAUČÍTE
- **Práce s genomovými daty** - FASTA, scaffoldy, sekvenční analýza

- **Pokročilý matplotlib** - subplots, gridspec, custom artists

- **Interaktivní vizualizace** - ipywidgets, plotly, bokeh

- **Bioinformatické algoritmy** - sliding windows, k-merová analýza

- **Výkonnostní optimalizace** - vektorizace, caching, profilování

# 🚀 TIPY PRO PRÁCI
- **Začněte malým datasetem** - extrahujte 1 scaffold pro testování

- **Pipeline přístup** - oddělte načítání, analýzu a vizualizaci

- **Cache výsledky** - ukládejte mezivýsledky do pickle/HDF5

- **Version control** - používejte Git pro správu kódu

- **AI jako parťák** - používejte pro debugging a optimalizace

**Deadline:** 7 dní
**Kontakt:** [mvohnoutova@jcu.cz]
**Konzultace:** [dny/hodiny]

"The goal is to transform data into information, and information into insight."
— Carly Fiorina

## Toto zadání je AI-odolné, protože:

- Vyžaduje kreativní návrh vizualizací

- Testuje porozumění algoritmů (3 implementace windows)

- Nutí interpretovat výsledky biologicky

- Hodnotí originalitu grafického výstupu

- Kombinuje více knihoven a technik

**Hotové řešení obsahuje:** Interaktivní dashboardy, publikovatelné grafy, genomový browser a pokročilé vizualizace, které jdou nad rámec jednoduchého vykreslení křivek.